In [ ]:
!pip install albumentations==1.4.0

In [ ]:
!cp -r /kaggle/input/test-rnsa-2024-code/Kaggle-2024-RSNA-Pub-main /kaggle/working/code

In [ ]:
import os
import json
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
kaggle_json = user_secrets.get_secret("kaggle")

os.makedirs("/root/.config/kaggle", exist_ok=True)

with open("/root/.config/kaggle/kaggle.json", "w") as f:
    f.write(kaggle_json)

os.chmod("/root/.config/kaggle/kaggle.json", 0o600)

In [ ]:
%cd /kaggle/working/code
%env PYTHONPATH=/kaggle/working/code:$PYTHONPATH

# !bash scripts/run.sh

In [ ]:
import os
import zipfile
import pathlib
from kaggle.api.kaggle_api_extended import KaggleApi

from src.utils import load_settings


def main():
    settings = load_settings()
    raw_data_dir = settings.raw_data_dir
    pretrained_checkpoint_dir = settings.pretrained_checkpoint_dir
    raw_data_dir.mkdir(parents=True, exist_ok=True)

    api = KaggleApi()
    api.authenticate()

    datasets = [
        {
            'type': 'dataset',
            'name': 'brendanartley/lumbar-coordinate-pretraining-dataset',
            'extract_dir': raw_data_dir / 'lumbar_coordinate_dataset'
        },
        {
            'type': 'competition_symlink',
            'name': 'rsna-2024-lumbar-spine-degenerative-classification',
            'extract_dir': raw_data_dir / 'rsna-2024-lumbar-spine-degenerative-classification'
        },
        ## NO MORE STORAGE :))))
        # {
        #     'type': 'dataset',
        #     'name': 'akinosora/rsna2024-lsdc-3rd-models-pub',
        #     'extract_dir': pretrained_checkpoint_dir,
        # },
    ]

    for item in datasets:
        if item['type'] == 'dataset':
            print(f"Downloading dataset {item['name']}")
            api.dataset_download_files(item['name'], path=str(raw_data_dir), quiet=False)
            zip_filename = item['name'].split('/')[-1] + '.zip'

            zip_file = raw_data_dir / zip_filename
            extract_dir = item['extract_dir']
            print(f'Extracting {zip_file} to {extract_dir}')
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            print(f'Removing {zip_file}')
            os.remove(zip_file)

        elif item['type'] == 'competition_symlink':
            src_path = pathlib.Path(f"/kaggle/input/{item['name']}")
            dst_path = item['extract_dir']

            if dst_path.exists():
                print(f"Already exists: {dst_path}")
            else:
                print(f"Linking competition dataset {src_path} -> {dst_path}")
                os.symlink(src_path, dst_path)


if __name__ == '__main__':
    main()


In [ ]:
# # !mkdir -p input/rsna-2024-lumbar-spine-degenerative-classification
# # !cp /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv input/rsna-2024-lumbar-spine-degenerative-classification/train.csv
# !mkdir -p input/rsna-2024-lumbar-spine-degenerative-classification
# !cp /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/*.csv input/rsna-2024-lumbar-spine-degenerative-classification/


In [ ]:
!ln -s /kaggle/input/rsna2024-lsdc-3rd-models-pub /kaggle/working/code/input/pretrained_checkpoint_dir

In [ ]:
!python src/utils/generate_kfold.py

In [ ]:
# !python3 src/stage1/moyashii/tools/create_dataset_v2.py moyashii 2024


In [ ]:
import os 
!mkdir -p /kaggle/working/code/input/processed

# Remove any old copies
!rm -rf /kaggle/working/code/input/processed/moyashii

# Copy
!cp -r /kaggle/input/projrsnadownload/code/input/processed/moyashii /kaggle/working/code/input/processed/


In [ ]:
# !rm -rf /kaggle/working/code/input/processed/moyashii
# !ln -s /kaggle/input/projrsnadownload/code/input/processed/moyashii /kaggle/working/code/input/processed/moyashii

# !rm -rf /kaggle/working/code/output/stage1/moyashii
# !ln -s /kaggle/input/projrsnadownload/code/output/stage1/moyashii /kaggle/working/code/output/stage1/moyashii


In [ ]:

# !rm -rf /kaggle/working/code/input/processed/moyashii
# !rm -rf /kaggle/working/code/output/stage1/moyashii


In [ ]:
!mkdir -p /kaggle/working/code/output/stage1/moyashii

!cp -r /kaggle/working/code/input/pretrained_checkpoint_dir/stage1/moyashii/* /kaggle/working/code/output/stage1/moyashii/


In [ ]:
# ### Moyashii
# # Create sagittal dataset v2 and train the sagittal keypoint models
# !python3 src/stage1/moyashii/tools/create_dataset_v2.py moyashii 2024
# !python3 src/stage1/moyashii/sagittal/train.py src/stage1/moyashii/sagittal/config/v2_0008_config.yaml 0008 --dst_root stage1/moyashii/v2/0008

In [ ]:
# # Create axial dataset v5 and train the axial keypoint models
# !python3 src/stage1/moyashii/tools/create_dataset_v5.py moyashii 2024
# !python3 src/stage1/moyashii/axial/train.py src/stage1/moyashii/axial/config/v5_0003_config.yaml 0003 --dst_root stage1/moyashii/v5/0003

In [ ]:
!cp -r /kaggle/input/test-rnsa-2024-code/Kaggle-2024-RSNA-Pub-main/input/processed/moyashii/keypoint_datasets/v6 /kaggle/working/code/input/processed/moyashii/keypoint_datasets/v6

In [ ]:
%cd /kaggle/working/code

# Create the dataset using keypoint detection models
!python3 src/stage2/moyashii/tools/create_dastaset_v9.py moyashii

In [ ]:
%cd /kaggle/working/code

# Train the center classification models (using dataset v9)
!python3 src/stage2/moyashii/train.py src/stage2/moyashii/config/0053.yaml 0053 --dst_root stage2/moyashii/center/v9/0053_42 --options folds=[0,1,2,3,4] seed=42
!python3 src/stage2/moyashii/train.py src/stage2/moyashii/config/0053.yaml 0053 --dst_root stage2/moyashii/center/v9/0053_2024 --options folds=[0,1,2,3,4] seed=2024


In [ ]:
%cd /kaggle/working/code

# Train the side classification models (using dataset v9)
!python3 src/stage2/moyashii/train.py src/stage2/moyashii/config/0054.yaml 0054 --dst_root stage2/moyashii/side/v9/0054_42 --options folds=[0,1,2,3,4] seed=42
!python3 src/stage2/moyashii/train.py src/stage2/moyashii/config/0054.yaml 0054 --dst_root stage2/moyashii/side/v9/0054_2024 --options folds=[0,1,2,3,4] seed=2024


In [ ]:
# # ### Moyashii
# # # Create sagittal dataset v2 and train the sagittal keypoint models
# # !python3 src/stage1/moyashii/tools/create_dataset_v2.py moyashii 2024
# # !python3 src/stage1/moyashii/sagittal/train.py src/stage1/moyashii/sagittal/config/v2_0008_config.yaml 0008 --dst_root stage1/moyashii/v2/0008

# # Create sagittal dataset v4 and train the sagittal keypoint models
# !python3 src/stage1/moyashii/tools/create_dataset_v4.py moyashii 2024
# !python3 src/stage1/moyashii/sagittal/train.py src/stage1/moyashii/sagittal/config/v4_0005_config.yaml 0005 --dst_root stage1/moyashii/v4/0005
# !python3 src/stage1/moyashii/sagittal/train.py src/stage1/moyashii/sagittal/config/v4_0012_config.yaml 0012 --dst_root stage1/moyashii/v4/0012
# # Create sagittal dataset v6 and train the sagittal keypoint models
# !python3 src/stage1/moyashii/tools/create_dataset_v6.py moyashii 2024
# !python3 src/stage1/moyashii/sagittal/train.py src/stage1/moyashii/sagittal/config/v6_0002_config.yaml 0002 --dst_root stage1/moyashii/v6/0002
# !python3 src/stage1/moyashii/sagittal/train.py src/stage1/moyashii/sagittal/config/v6_0003_config.yaml 0003 --dst_root stage1/moyashii/v6/0003
# # Create axial dataset v5 and train the axial keypoint models
# !python3 src/stage1/moyashii/tools/create_dataset_v5.py moyashii 2024
# !python3 src/stage1/moyashii/axial/train.py src/stage1/moyashii/axial/config/v5_0003_config.yaml 0003 --dst_root stage1/moyashii/v5/0003



In [ ]:
%cd /kaggle/working/code
!python3 src/predict.py